In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import planet_ruler.observation as pro

from planet_ruler.demo import make_dropdown, load_demo_parameters, display_text
from planet_ruler.plot import plot_3d_solution
from planet_ruler.dashboard import OutputCapture

# Choose a Target

Run the next cell and choose a target from the dropdown menu.

In [ ]:
demo = make_dropdown()
display(demo)

Load presets using the next cell and then step through the rest of the demo in order.

In [ ]:
demo_params = load_demo_parameters(demo)

In [ ]:
demo_params

# Load an Image

In [ ]:
Obs = pro.LimbObservation(
    image_filepath=demo_params["image_filepath"], fit_config=demo_params["fit_config"]
)

In [ ]:
display_text(demo_params["preamble"])

Obs.plot()

# Detect Limb

Next we want to define where the limb is so we can start figuring out the radius. There are many ways to go about this. For example, we could look for regions of high contrast, draw by hand, invent an algorithm, or just use some pre-trained ML. 

For this demo you have the option to use a recent (probably not by the time you read this) pre-trained segmentation model -- 'Segment Anything', or to annotate the limb by hand. The ML route is more automatic but it requires lots of compute power and is more easily fooled than your brain. Feel free to choose either route, or use both and compare!

### Manual Detection

In [ ]:
# run this cell if you want to annotate the limb by hand
Obs.limb_detection = "manual"
Obs.detect_limb()

### Automatic Detection

In [ ]:
# run this cell if you want to annotate the limb using ML
Obs.limb_detection = "segmentation"
Obs.detect_limb()
Obs.smooth_limb(method="rolling-median", window_length=1)

We can now take a look at the fitted/annotated limb location.

In [ ]:
Obs.plot()

Let's add a save point in case we don't want to do all that again.

In [ ]:
Obs.save_limb(demo_params["limb_save"])

# Fit Planet Radius

In [ ]:
display_text(demo_params["parameter_walkthrough"])

In [ ]:
# Set n_jobs to the number of processors you feel comfortable utilizing.
N_JOBS = 6

# Create capture
capture = OutputCapture(max_lines=20, line_width=70)

# Use with context manager
with capture:
    Obs.fit_limb(
        seed=0,
        minimizer="differential-evolution",
        n_jobs=N_JOBS,
        verbose=True,
        dashboard=True,
        dashboard_kwargs={
            "output_capture": capture,
            "width": 80,
            "max_warnings": 5,
            "max_hints": 4,
            "min_message_display_time": 5.0,
        },
    )

## Check Fit

### By Eye

Let's take a look at the solution and see if it makes sense. It's not challenging to end up with a fit that doesn't converge well and appears to have nothing to do with the limb in question. If that happens, don't despair! If the limb detection went okay, it usually it means some tweaking of the parameter ranges or optimizer properties is in order.

We can start by looking at the predicted limb given our optimal parameters. If any of these looks wrong, it likely is. Check your posteriors in the next section and you can try to tweak things.

In [ ]:
Obs.plot()

We can zoom out a little to see how what the image captures compares to the rest of the planet. There is a lot to that iceberg!

Zooming out even further gives us a real view to what is going on here in three dimensions.

In [ ]:
plot_3d_solution(**Obs.best_parameters)

### Posteriors

To understand this section you need to know a bit about what minimizer we used to solve the problem. This is using something called [differential evolution](https://en.wikipedia.org/wiki/Differential_evolution). It's a (terribly named) way of minimizing a loss function that is non-differentiable. Basically it simulates the 'evolution' of a population composed of parameter value sets. So [r=1, h=2, etc.] could be one unit of the population. These units are mutated and combined over many generations where only the best-fitting survive. When we end the simulation we have an optimal parameter set and a population that is hopefully still competitive. We can think of that population as something like a 'posterior', or measure of uncertainty in each parameter's observed value. Another totally valid method of optimization would have been a Bayesian MCMC and it would also give us posterior distributions that we could check out.

Below we plot all these populations/posteriors and see if they look good. What is good? Ideally they should form a concave U-shape around the best value and not be too much up against either of the limits we imposed in the fit. If the latter is taking place, we might not be reaching the true minimum and it's hard to take the results too seriously. In that case, try the fit again but move the constraints out a little to give the fit some breathing room. I say a little bit because you don't want to make the constraints too wide either -- there really is a sweet spot that doesn't explicitly block the best parameter values, but gives the minimizer enough to work with that it can arrive there in our lifetimes.

I already went through the trial and error to get the following posteriors to look right -- if implementing the code on a new image, take these an example of what to shoot for.

In [ ]:
pro.plot_diff_evol_posteriors(Obs, show_points=True, log=False)

If you prefer hard numbers, here's a summary.

In [ ]:
pro.unpack_diff_evol_posteriors(Obs).describe()

# The Answer!

At long last. After convincing ourselves that we can trust our results, let's take a look.

In [ ]:
# Get radius uncertainty (automatic method selection)
print(f"Radius: {Obs.radius_km:.1f} ± {Obs.radius_uncertainty:.1f} km")

# Get uncertainty for any parameter
altitude_unc = Obs.parameter_uncertainty("h", scale_factor=1e-3)
print(f"Altitude: {Obs.altitude_km:.1f} ± {altitude_unc['uncertainty']:.1f} km")
print(f"Method used: {altitude_unc['method']}")

pct_error = (
    (Obs.radius_km - demo_params["true_radius"]) / demo_params["true_radius"] * 100
)

print(
    f"Our estimate of {demo_params['target']}'s radius is{Obs.radius_km: .1f} \u00b1{Obs.radius_uncertainty: .0f} kilometers."
)
print(
    f"That puts us {pct_error: .1f}% away from the true value of {demo_params['true_radius']} km."
)

In [ ]:
pluto_limb = Obs.features["fitted_limb"]
np.save("../pluto_limb.npy", pluto_limb)

The value is quite close to the truth. I wouldn't plan a space trip around it but it isn't bad for a single image and rough idea of what camera was used!

See below where all the other parameters ended up relative to their initial values.

In [ ]:
pro.package_results(Obs)